In [1]:
import os
import glob
import re
import numpy as np
from osgeo import gdal
import struct

In [43]:
targetX=[-120.22, -120.22, -119.19, -119.19]
targetY=[36.24,   35.74,  35.74,  36.24]
#geoinfo = '\"35.88 36.11 -119.89 -119.64\"'
geoinfo = '\"35.73 36.25 -120.23 -119.18\"'
geodir='/home/jovyan/CV/merged/geom_reference' #full path to merged interferogram directory
intname='/home/jovyan/CV/merged/interferograms/20190814_20190820/fine.int'

In [44]:
rowname=geodir+'/rows.rdr'
colname=geodir+'/cols.rdr'

# Need to fix the xml and vrt files for the cfloat->float

In [45]:
rds = gdal.Open(intname)
img_width,img_height=rds.RasterXSize,rds.RasterYSize

print(int(img_width))
print(int(img_height))

2426
1837


In [46]:
os.system('cp '+intname+'.xml '+'tmp.xml')

suffs = ('.vrt','.xml')
files = (rowname,colname)
for suff in suffs:
    for file in files:
        os.system('cp '+intname+suff+' '+file+suff)

        #input file
        fin= open(file+suff, "rt")
        data = fin.read()
        data = data.replace('CFloat', 'Float')      
        data = data.replace('CFLOAT', 'FLOAT')      
        data = data.replace('<PixelOffset>8</PixelOffset>','<PixelOffset>4</PixelOffset>')
        data = data.replace(intname,file)
        data = data.replace('fine.int',file)
        data = data.replace('<LineOffset>'+str(img_width*8)+'</LineOffset>','<LineOffset>'+str(img_width*4)+'</LineOffset>')
        data = data.replace('cpx','FLOAT')
        fin.close()
    
        fout = open(file+suff,'wt')
        fout.write(data)
        fout.close()



In [47]:
filecol = open(colname, "wb")
filerow = open(rowname, "wb")

for i in np.arange(img_height):
    for j in np.arange(img_width):
        filerow.write(struct.pack('<f',i+1))
        filecol.write(struct.pack('<f',j+1))

filerow.close()
filecol.close()

In [48]:
os.system("fixImageXml.py -i "+geodir+"/rows.rdr -f")
os.system("fixImageXml.py -i "+geodir+"/cols.rdr -f")

This is the Open Source version of ISCE.
Some of the workflows depend on a separate licensed package.
To obtain the licensed package, please make a request for ISCE
through the website: https://download.jpl.nasa.gov/ops/request/index.cfm.
Alternatively, if you are a member, or can become a member of WinSAR
you may be able to obtain access to a version of the licensed sofware at
https://winsar.unavco.org/software/isce
Writing geotrans to VRT for /home/jovyan/CV/merged/geom_reference/rows.rdr
This is the Open Source version of ISCE.
Some of the workflows depend on a separate licensed package.
To obtain the licensed package, please make a request for ISCE
through the website: https://download.jpl.nasa.gov/ops/request/index.cfm.
Alternatively, if you are a member, or can become a member of WinSAR
you may be able to obtain access to a version of the licensed sofware at
https://winsar.unavco.org/software/isce
Writing geotrans to VRT for /home/jovyan/CV/merged/geom_reference/cols.rdr


0

In [49]:
print('run these from the directory where the row/col.rdr files are sitting, within merged/geom_reference')
command='geocodeGdal.py --lat /home/jovyan/CV/merged/geom_reference/lat.rdr --lon /home/jovyan/CV/merged/geom_reference/lon.rdr -r nearest -b '+geoinfo+' -x 0.0001 -y 0.0001 -f rows.rdr'
print(command)
  
command='geocodeGdal.py --lat /home/jovyan/CV/merged/geom_reference/lat.rdr --lon /home/jovyan/CV/merged/geom_reference/lon.rdr -r nearest -b '+geoinfo+' -x 0.0001 -y 0.0001 -f cols.rdr'
print(command)
                                  

run these from the directory where the row/col.rdr files are sitting, within merged/geom_reference
geocodeGdal.py --lat /home/jovyan/CV/merged/geom_reference/lat.rdr --lon /home/jovyan/CV/merged/geom_reference/lon.rdr -r nearest -b "35.73 36.25 -120.23 -119.18" -x 0.0001 -y 0.0001 -f rows.rdr
geocodeGdal.py --lat /home/jovyan/CV/merged/geom_reference/lat.rdr --lon /home/jovyan/CV/merged/geom_reference/lon.rdr -r nearest -b "35.73 36.25 -120.23 -119.18" -x 0.0001 -y 0.0001 -f cols.rdr


# Run the two commands above in a terminal WITHIN the merged/geom_reference directory

In [50]:
target_row=np.empty((4));
target_col=np.empty((4));
target_col[:]=np.NaN;
target_row[:]=np.NaN;
for i in np.arange(4):

    ds = gdal.Open('/home/jovyan/CV/merged/geom_reference/geo_rows.rdr')
    ulx, xres, xskew, uly, yskew, yres  = ds.GetGeoTransform()

    px = np.round((targetX[i]-ulx) / xres)
    py = np.round((targetY[i]-uly) / yres)

    if px <= 0 or px > ds.RasterXSize:
        print(str(targetX[i])+' out of range: px='+str(px))
    elif py <=0 or py > ds.RasterYSize:
        print(str(targetY[i])+' out of range: py='+str(py))
    else:
        tmpval = ds.ReadRaster(int(px), int(py), 1,1, buf_type = gdal.GDT_Float32)
        tmpval = struct.unpack('f' ,tmpval)
        target_row[i] = tmpval[0] #because struct unpack returns a tuple
        ds=None
        ds = gdal.Open('/home/jovyan/CV/merged/geom_reference/geo_cols.rdr')
        tmpval = ds.ReadRaster(int(px), int(py), 1,1, buf_type = gdal.GDT_Float32)
        tmpval = struct.unpack('f' ,tmpval)
        target_col[i] = tmpval[0] #because struct unpack returns a tuple
        ds=None

print('target row is '+str(target_row)+' target col is '+str(target_col))

target row is [ 619. 1602. 1315.  336.] target col is [1668. 1525.  260.  390.]
